# Set up environment

In [ ]:
%pip install torch torchvision torchaudio
%pip install transformers
%pip install groq
%pip install transformers sentencepiece accelerate
%pip install -q transformers accelerate bitsandbytes sentencepiece torch
%pip install openai
%pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.5 MB/s eta 0:00:00


In [ ]:
import os
#Reserved_groq_token

os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY"
OWNER = "Springblade"
REPO = "Project-LLM"
ROOT_PATH = "extracted_data"

TOKEN = "github_token"   # <<< PUT TOKEN HERE

HEADERS = {"Authorization": f"token {TOKEN}"}

API_URL = f"https://api.github.com/repos/{OWNER}/{REPO}/contents/{ROOT_PATH}"

#Preprocessing data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Extracting text data from github

In [ ]:
import os, json

def save_dict_to_json(file_name: str, data: dict, base_dir: str = "/content/drive/MyDrive/TextSummary"):

    # Ensure the file name has .json extension
    if not file_name.endswith(".json"):
        file_name += ".json"

    # Construct full path
    full_path = os.path.join(base_dir, file_name)

    # Create directories if needed
    os.makedirs(os.path.dirname(full_path), exist_ok=True)

    # Save dictionary as JSON
    with open(full_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"JSON saved successfully to: {full_path}")


In [ ]:
%pip install json5

In [ ]:
import json
import re
import json5


def extract_json_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 1. Try normal JSON first
    try:
        return json.loads(content)
    except Exception as e:
        print(f"[!] Standard JSON load failed: {e}")

    # ----------------------------------------------------------------------
    # 2. Repair common issues
    repaired = content

    # (A) Fix fancy quotes “ ” ‘ ’
    repaired = repaired.replace("“", "\"").replace("”", "\"")
    repaired = repaired.replace("‘", "'").replace("’", "'")

    # (B) Remove trailing commas before } or ]
    repaired = re.sub(r",\s*([}\]])", r"\1", repaired)

    # (C) Escape unescaped inner quotes inside strings like:
    # "Specific": "something "quoted" inside"
    # Only fix quotes that appear between letters
    repaired = re.sub(r'(?<=\w)"(?=\w)', r'\\"', repaired)

    # (D) Remove invalid control characters
    repaired = re.sub(r"[\x00-\x1F]+", " ", repaired)

    # 3. Try loading again after repairs
    try:
        return json.loads(repaired)
    except Exception as e:
        print(f"[!] Repaired JSON load still failed: {e}")

    # ----------------------------------------------------------------------
    # 4. FINAL FALLBACK: json5 (very tolerant)
    try:
        print("[✓] Using json5 fallback (tolerant parser)")
        return json5.loads(content)
    except Exception as e:
        print(f"[!] Even json5 failed: {e}")

    raise ValueError("Unable to parse JSON after all recovery attempts.")


#Example usage
#extracted_batch1 =extract_json_from_file("/content/drive/MyDrive/TextSummary/Batch1Text.json")

In [ ]:
import requests
API_URL = f"https://api.github.com/repos/{OWNER}/{REPO}/contents/{ROOT_PATH}"

def get_md_files(url, md_files):
    response = requests.get(url, headers=HEADERS)
    items = response.json()

    # Handle GitHub error cleanly
    if isinstance(items, dict) and items.get("message"):
        print("GitHub API error:", items["message"])
        return md_files   # return what we have instead of None

    for item in items:
        if item["type"] == "file" and item["name"].endswith(".md"):
            raw_url = item["download_url"]
            text = requests.get(raw_url, headers=HEADERS).text
            md_files[item["path"]] = text

        elif item["type"] == "dir":
            get_md_files(item["url"], md_files)

    return md_files

Data extraction result here

In [ ]:
all_md = get_md_files(API_URL, {})

print("Downloaded:", len(all_md), "markdown files")
for path, content in all_md.items():
    print("----", path, "----")
    print(content[:200], "...")  # preview first 200 chars

Downloaded: 93 markdown files
---- extracted_data/1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi/1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi.md ----
1

# A 20-Year-Old Woman from Sudan With Fever, Haemorrhage and Shock

DANIEL G. BAUSCH

# Clinical Presentation

# History

A 20-year-old housewife presents to a hospital in northern Uganda with a 2- ...
---- extracted_data/10---A-55-Year-Old-Indigenous-Woman-from-Australia-W_2022_Clinical-Cases-in-/10---A-55-Year-Old-Indigenous-Woman-from-Australia-W_2022_Clinical-Cases-in-.md ----
# 10

## A 55-Year-Old Indigenous Woman from Australia With a Widespread Exfoliating Rash and Sepsis

BART J. CURRIE AND JAMES MCCARTHY

#### Clinical Presentation

#### History

You are working in a  ...
---- extracted_data/11---A-45-Year-Old-Male-Security-Guard-from-Malawi-_2022_Clinical-Cases-in-T/11---A-45-Year-Old-Male-Security-Guard-from-Malawi-_2022_Clinical-Cases-in-T.md ----
# 11

# A 45-Y

In [ ]:
len(all_md)

93

In [ ]:
#particion the orginal dictionary to prevent wasting tokens if error happen
def chunk_dict_contiguous(d, parts=4):
    items = list(d.items())
    n = len(items)
    q, r = divmod(n, parts)
    result = []
    start = 0
    for i in range(parts):
        size = q + (1 if i < r else 0)   # distribute remainder to first r parts
        chunk_items = items[start:start+size]
        result.append(dict(chunk_items))
        start += size
    return result

# Example
#d = {f"k{i}": i for i in range(11)}
#a, b, c, d4 = chunk_dict_contiguous(d, 4)
#print(len(a), len(b), len(c), len(d4))  # -> balanced sizes like 3,3,3,2

In [ ]:
batch1, batch2, batch3, batch4 = chunk_dict_contiguous(all_md, 4)

Feature extraction here

In [ ]:
from openai import OpenAI
openai_client = OpenAI(api_key=os.environ["GROQ_API_KEY"], base_url="https://api.groq.com/openai/v1")

def run_llama(prompt, model="llama-3.1-8b-instant",pre_prompt_instruction='You are a help medical assistant'):

    completion = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": pre_prompt_instruction},
            {"role": "user", "content": prompt},
        ],
    )
    return completion.choices[0].message.content

In [ ]:
user_pre_prompt_instruction = """
           You are TropID-Extractor, an expert clinical information extractor for tropical & infectious diseases.  ,
           Your task is to create a short descripition about the disease based on provided document and your base knowledge. The output must follow below json format, no comment or deviation from instructed format:
           {
                "Disease_summary": "Sufficient information about the disease includes primary area of occurance (optional), symptom (mandatory), clinical presentation (optional) , differential diagnosis (manadatory if it is included in the context), key finding to diagnose this exact disease (Lab test, scanning result, blood test, etc,mandatory)",
                "Keyword": "Name of the disease, short tag to identify the disease"
           }
           Example output
           {
                "Disease_summary": "Ebola virus disease primarily occurs in sub-Saharan Africa and typically presents with acute fever, severe headache, myalgia, vomiting, and diarrhea, with hemorrhagic manifestations in severe cases. Clinically, it is characterized by rapid progression to gastrointestinal illness, dehydration, and multiorgan dysfunction. Important differential diagnoses include malaria, typhoid fever, Lassa fever, and dengue hemorrhagic fever. The diagnosis is confirmed by detection of Ebola virus RNA using RT-PCR from blood or other body fluids, supported by relevant epidemiological exposure.",
                "Keyword": "Ebola"
           }
          """


In [ ]:
from openai import OpenAI, RateLimitError, APIError
import time

def summarize_text(Batch: dict, pre_prompt_instruction: str = ''):
    outputSummarised = {}
    error_list = {}
    prompt_template = (
        "Now do the assigned task from this clinical case:"
    )

    for file_name, raw_text in Batch.items():
        print(f"Working with {file_name}\n")
        user_prompt = f"{prompt_template}\nCase report:\n{raw_text}"

        try:
            summary = run_llama(
                    user_prompt,
                    pre_prompt_instruction=pre_prompt_instruction
                )
            outputSummarised[file_name] = summary

        except Exception as e:
            print(f"Unexpected error on {file_name}: {e}. Skipping this iteration")
            error_list[file_name] = raw_text


    return outputSummarised,error_list


In [ ]:
batch1_summary,error_list = summarize_text(batch1,pre_prompt_instruction = user_pre_prompt_instruction)

Working with extracted_data/1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi/1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi.md

Working with extracted_data/10---A-55-Year-Old-Indigenous-Woman-from-Australia-W_2022_Clinical-Cases-in-/10---A-55-Year-Old-Indigenous-Woman-from-Australia-W_2022_Clinical-Cases-in-.md

Working with extracted_data/11---A-45-Year-Old-Male-Security-Guard-from-Malawi-_2022_Clinical-Cases-in-T/11---A-45-Year-Old-Male-Security-Guard-from-Malawi-_2022_Clinical-Cases-in-T.md

Working with extracted_data/12---A-29-Year-Old-Man-from-The-Gambia-With-G_2022_Clinical-Cases-in-Tropica/12---A-29-Year-Old-Man-from-The-Gambia-With-G_2022_Clinical-Cases-in-Tropica.md

Working with extracted_data/13---A-16-Year-Old-Girl-from-Malawi-With-Fever_2022_Clinical-Cases-in-Tropic/13---A-16-Year-Old-Girl-from-Malawi-With-Fever_2022_Clinical-Cases-in-Tropic.md

Working with extracted_data/14---A-22-Year-Old-Woman-from-Bangladesh-W

In [ ]:
batch2_summary,error_list_2 = summarize_text(batch2,pre_prompt_instruction = user_pre_prompt_instruction)

Working with extracted_data/32---A-44-Year-Old-Male-Farmer-from-Laos-With-Di_2022_Clinical-Cases-in-Trop/32---A-44-Year-Old-Male-Farmer-from-Laos-With-Di_2022_Clinical-Cases-in-Trop.md

Working with extracted_data/33---A-53-Year-Old-Man-from-Malawi-With-a-C_2022_Clinical-Cases-in-Tropical-/33---A-53-Year-Old-Man-from-Malawi-With-a-C_2022_Clinical-Cases-in-Tropical-.md

Working with extracted_data/34---A-35-Year-Old-Male-Farmer-from-Peru-With-a-Chro_2022_Clinical-Cases-in-/34---A-35-Year-Old-Male-Farmer-from-Peru-With-a-Chro_2022_Clinical-Cases-in-.md

Working with extracted_data/35---A-32-Year-Old-Woman-from-Malawi-With-Heada_2022_Clinical-Cases-in-Tropi/35---A-32-Year-Old-Woman-from-Malawi-With-Heada_2022_Clinical-Cases-in-Tropi.md

Working with extracted_data/36---A-23-Year-Old-Farmer-from-Myanmar-With-Uni_2022_Clinical-Cases-in-Tropi/36---A-23-Year-Old-Farmer-from-Myanmar-With-Uni_2022_Clinical-Cases-in-Tropi.md

Working with extracted_data/37---A-29-Year-Old-Woman-from-Malawi-With-

In [ ]:
batch3_summary,error_list_3 = summarize_text(batch3,pre_prompt_instruction = user_pre_prompt_instruction)

Working with extracted_data/53---A-24-Year-Old-Woman-from-Uganda-With-Fe_2022_Clinical-Cases-in-Tropical/53---A-24-Year-Old-Woman-from-Uganda-With-Fe_2022_Clinical-Cases-in-Tropical.md

Working with extracted_data/54---A-52-Year-Old-Male-Safari-Tourist-Returning-fro_2022_Clinical-Cases-in-/54---A-52-Year-Old-Male-Safari-Tourist-Returning-fro_2022_Clinical-Cases-in-.md

Working with extracted_data/55---A-40-Year-Old-Male-Farmer-from-Peru-With-Ch_2022_Clinical-Cases-in-Trop/55---A-40-Year-Old-Male-Farmer-from-Peru-With-Ch_2022_Clinical-Cases-in-Trop.md

Working with extracted_data/56---A-21-Year-Old-Pregnant-Woman-from-The-Ga_2022_Clinical-Cases-in-Tropica/56---A-21-Year-Old-Pregnant-Woman-from-The-Ga_2022_Clinical-Cases-in-Tropica.md

Working with extracted_data/57---A-37-Year-Old-Woman-from-Malawi-With-H_2022_Clinical-Cases-in-Tropical-/57---A-37-Year-Old-Woman-from-Malawi-With-H_2022_Clinical-Cases-in-Tropical-.md

Working with extracted_data/58---A-25-Year-Old-Woman-from-Egypt-With-S

In [ ]:
batch4_summary,error_list_4 = summarize_text(batch4,pre_prompt_instruction = user_pre_prompt_instruction)

Working with extracted_data/74---A-28-Year-Old-Woman-from-Sierra-Leone-With-_2022_Clinical-Cases-in-Trop/74---A-28-Year-Old-Woman-from-Sierra-Leone-With-_2022_Clinical-Cases-in-Trop.md

Working with extracted_data/75---A-25-Year-Old-Woman-from-Zambia-With-a-N_2022_Clinical-Cases-in-Tropica/75---A-25-Year-Old-Woman-from-Zambia-With-a-N_2022_Clinical-Cases-in-Tropica.md

Working with extracted_data/76---A-55-Year-Old-Woman-from-Turkey-With-Feve_2022_Clinical-Cases-in-Tropic/76---A-55-Year-Old-Woman-from-Turkey-With-Feve_2022_Clinical-Cases-in-Tropic.md

Working with extracted_data/77---A-51-Year-Old-Female-Traveller-Returning-from-Cen_2022_Clinical-Cases-i/77---A-51-Year-Old-Female-Traveller-Returning-from-Cen_2022_Clinical-Cases-i.md

Working with extracted_data/78---A-42-Year-Old-British-Man-Living-in-Malawi_2022_Clinical-Cases-in-Tropi/78---A-42-Year-Old-British-Man-Living-in-Malawi_2022_Clinical-Cases-in-Tropi.md

Working with extracted_data/79---A-34-Year-Old-Male-Immigrant-from-Per

In [ ]:
save_dict_to_json("Batch1",batch1_summary)
save_dict_to_json("Batch2",batch2_summary)
save_dict_to_json("Batch3",batch3_summary)
save_dict_to_json("Batch4",batch4_summary)

JSON saved successfully to: /content/drive/MyDrive/TextSummary/Batch1.json
JSON saved successfully to: /content/drive/MyDrive/TextSummary/Batch2.json
JSON saved successfully to: /content/drive/MyDrive/TextSummary/Batch3.json
JSON saved successfully to: /content/drive/MyDrive/TextSummary/Batch4.json


In [ ]:
extracted_batch1 =extract_json_from_file("/content/drive/MyDrive/TextSummary/Batch1.json")
extracted_batch2 =extract_json_from_file("/content/drive/MyDrive/TextSummary/Batch2.json")
extracted_batch3 =extract_json_from_file("/content/drive/MyDrive/TextSummary/Batch3.json")
extracted_batch4 =extract_json_from_file("/content/drive/MyDrive/TextSummary/Batch4.json")

Working with image

In [ ]:
import json
import requests

url = "https://raw.githubusercontent.com/Springblade/Project-LLM/main/images_metadata.json"
data = requests.get(url).json()

In [ ]:
from io import BytesIO
from PIL import Image
def extract_image_modal(data):
  url_path = []
  caption = []
  source_file = []
  pil_images = []
  for image_id, json_val in data.items():
    url_link = json_val.get("Path")
    url_path.append(url_link)
    caption.append(json_val.get("Caption"))
    source_file.append(json_val.get("SourceFile"))
    response = requests.get(url_link)
    pil_images.append(Image.open(BytesIO(response.content)).convert("RGB"))
  return url_path, caption, source_file, pil_images

In [ ]:
url_path, caption, source_file, pil_images = extract_image_modal(data)

# Embedding data


In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoProcessor
from PIL import Image
import torch
import numpy as np
import base64, io

# Load Jina CLIP model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-clip-v1")
model = AutoModel.from_pretrained("jinaai/jina-clip-v1")
processor = AutoProcessor.from_pretrained("jinaai/jina-clip-v1")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-flash-implementation:
- embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-flash-implementation:
- modeling_bert.py
- block.py
- bert_padding.py
- embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


preprocessor_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

jinaai/jina-clip-implementation You can inspect the repository content at https://hf.co/jinaai/jina-clip-v1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


processing_clip.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-clip-implementation:
- processing_clip.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
#Explicitly extract disease name for metadata
#disease_name = "Unknown"
#json_objects = cleaned_output
#section_name = []
#textual_info = []
#for section in json_objects:
#  for key, value in section.items():
#    if key == 'disease_name':
#      disease_name = value
#disease_name

'Ebola'

In [ ]:
extracted_batch2

{'extracted_data/32---A-44-Year-Old-Male-Farmer-from-Laos-With-Di_2022_Clinical-Cases-in-Trop/32---A-44-Year-Old-Male-Farmer-from-Laos-With-Di_2022_Clinical-Cases-in-Trop.md': '{\n    "Disease_summary": "Melioidosis is an infectious disease caused by B. pseudomallei, prevalent in South and South-east Asia and northern Australia, characterized by fever, headache, myalgia, arthralgia, and a painful abscess over the left scapula. It is often associated with immunocompromised patients, especially those with diabetes mellitus. The disease ranges from benign localized infection to rapidly progressive septicaemia. Bacteraemic form accounts for 40-60% of cases, and over half present with pneumonia, which carries a case fatality rate of 10-40%. Treatment involves parenteral antibiotics for 2 weeks, followed by oral eradication therapy for 3-6 months.",\n    "Keyword": "Melioidosis"\n}',
 'extracted_data/33---A-53-Year-Old-Man-from-Malawi-With-a-C_2022_Clinical-Cases-in-Tropical-/33---A-53-Year-

In [ ]:
import json
import re
import numpy as np
import torch

def safe_json_loads(text, file_name=None):
    """Try to safely parse JSON-like text produced by LLMs."""
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # Attempt common fixes
        fixed = text.strip()

        # Replace single quotes with double quotes (careful with numbers)
        fixed = fixed.replace("'", '"')

        # Remove trailing commas before } or ]
        fixed = re.sub(r",(\s*[}\]])", r"\1", fixed)

        # Ensure it starts/ends correctly
        if not fixed.startswith("{"):
            fixed = "{" + fixed
        if not fixed.endswith("}"):
            fixed = fixed + "}"

        try:
            return json.loads(fixed)
        except Exception as e:
            print(f"⚠️ Failed to parse JSON for {file_name or 'unknown file'}: {e}")
            return None


def embed_text(ALL_Text_Extraction):
    context, hashtag = [], []
    embedded_context, embedded_hashtag = [], []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Running on device: {device}")

    for file_name, text in ALL_Text_Extraction.items():
        print(f"Working with {file_name}")
        json_objects = safe_json_loads(text, file_name=file_name)
        if json_objects is None:
            continue

        ctx = json_objects["Disease_summary"]
        tag = json_objects["Keyword"]

        context.append(ctx)
        hashtag.append(tag)

        # ---- context embedding ----
        inputs = tokenizer(
            ctx,
            return_tensors="pt",
            truncation=True,
            max_length=12000,
        ).to(device)

        with torch.no_grad():
            feat = model.get_text_features(**inputs)

        feat = feat[0]  # ✅ remove batch dimension
        feat = feat / feat.norm(p=2)
        embedded_context.append(feat.cpu().tolist())

        # ---- hashtag embedding ----
        inputs = tokenizer(
            tag,
            return_tensors="pt",
            truncation=True,
            max_length=12000,
        ).to(device)

        with torch.no_grad():
            feat = model.get_text_features(**inputs)

        feat = feat[0]  # ✅ remove batch dimension
        feat = feat / feat.norm(p=2)
        embedded_hashtag.append(feat.cpu().tolist())

    return context, embedded_context, hashtag, embedded_hashtag


In [ ]:
import json
import re
import numpy as np
import torch
def embed_plain_text(text_string):
    """
    Embed a plain list of text strings using Jina CLIP v1.
    """
    # Prefer GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Tokenize text
    inputs = tokenizer(
        text_string,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=3000  # typical safe limit for plain text
    ).to(device)

    # Compute embeddings
    with torch.no_grad():
        text_features = model.get_text_features(**inputs)

    # Normalize embeddings (cosine similarity compatible)
    text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)

    # Convert to numpy
    return text_features.cpu().numpy()


In [ ]:
context1, embedded_context1, hashtag1, embedded_hashtag1 = embed_text(extracted_batch1)

Running on device: cuda
Working with extracted_data/1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi/1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi.md
Working with extracted_data/10---A-55-Year-Old-Indigenous-Woman-from-Australia-W_2022_Clinical-Cases-in-/10---A-55-Year-Old-Indigenous-Woman-from-Australia-W_2022_Clinical-Cases-in-.md
Working with extracted_data/11---A-45-Year-Old-Male-Security-Guard-from-Malawi-_2022_Clinical-Cases-in-T/11---A-45-Year-Old-Male-Security-Guard-from-Malawi-_2022_Clinical-Cases-in-T.md
Working with extracted_data/12---A-29-Year-Old-Man-from-The-Gambia-With-G_2022_Clinical-Cases-in-Tropica/12---A-29-Year-Old-Man-from-The-Gambia-With-G_2022_Clinical-Cases-in-Tropica.md
Working with extracted_data/13---A-16-Year-Old-Girl-from-Malawi-With-Fever_2022_Clinical-Cases-in-Tropic/13---A-16-Year-Old-Girl-from-Malawi-With-Fever_2022_Clinical-Cases-in-Tropic.md
Working with extracted_data/14---A-22-Year-Old-Woma

In [ ]:
context2, embedded_context2, hashtag2, embedded_hashtag2 = embed_text(extracted_batch2)

Running on device: cuda
Working with extracted_data/32---A-44-Year-Old-Male-Farmer-from-Laos-With-Di_2022_Clinical-Cases-in-Trop/32---A-44-Year-Old-Male-Farmer-from-Laos-With-Di_2022_Clinical-Cases-in-Trop.md
Working with extracted_data/33---A-53-Year-Old-Man-from-Malawi-With-a-C_2022_Clinical-Cases-in-Tropical-/33---A-53-Year-Old-Man-from-Malawi-With-a-C_2022_Clinical-Cases-in-Tropical-.md
Working with extracted_data/34---A-35-Year-Old-Male-Farmer-from-Peru-With-a-Chro_2022_Clinical-Cases-in-/34---A-35-Year-Old-Male-Farmer-from-Peru-With-a-Chro_2022_Clinical-Cases-in-.md
Working with extracted_data/35---A-32-Year-Old-Woman-from-Malawi-With-Heada_2022_Clinical-Cases-in-Tropi/35---A-32-Year-Old-Woman-from-Malawi-With-Heada_2022_Clinical-Cases-in-Tropi.md
Working with extracted_data/36---A-23-Year-Old-Farmer-from-Myanmar-With-Uni_2022_Clinical-Cases-in-Tropi/36---A-23-Year-Old-Farmer-from-Myanmar-With-Uni_2022_Clinical-Cases-in-Tropi.md
Working with extracted_data/37---A-29-Year-Old-Woma

In [ ]:
context3, embedded_context3, hashtag3, embedded_hashtag3 = embed_text(extracted_batch3)

Running on device: cuda
Working with extracted_data/53---A-24-Year-Old-Woman-from-Uganda-With-Fe_2022_Clinical-Cases-in-Tropical/53---A-24-Year-Old-Woman-from-Uganda-With-Fe_2022_Clinical-Cases-in-Tropical.md
Working with extracted_data/54---A-52-Year-Old-Male-Safari-Tourist-Returning-fro_2022_Clinical-Cases-in-/54---A-52-Year-Old-Male-Safari-Tourist-Returning-fro_2022_Clinical-Cases-in-.md
Working with extracted_data/55---A-40-Year-Old-Male-Farmer-from-Peru-With-Ch_2022_Clinical-Cases-in-Trop/55---A-40-Year-Old-Male-Farmer-from-Peru-With-Ch_2022_Clinical-Cases-in-Trop.md
Working with extracted_data/56---A-21-Year-Old-Pregnant-Woman-from-The-Ga_2022_Clinical-Cases-in-Tropica/56---A-21-Year-Old-Pregnant-Woman-from-The-Ga_2022_Clinical-Cases-in-Tropica.md
Working with extracted_data/57---A-37-Year-Old-Woman-from-Malawi-With-H_2022_Clinical-Cases-in-Tropical-/57---A-37-Year-Old-Woman-from-Malawi-With-H_2022_Clinical-Cases-in-Tropical-.md
Working with extracted_data/58---A-25-Year-Old-Woma

In [ ]:
context4, embedded_context4, hashtag4, embedded_hashtag4 = embed_text(extracted_batch4)

Running on device: cuda
Working with extracted_data/74---A-28-Year-Old-Woman-from-Sierra-Leone-With-_2022_Clinical-Cases-in-Trop/74---A-28-Year-Old-Woman-from-Sierra-Leone-With-_2022_Clinical-Cases-in-Trop.md
Working with extracted_data/75---A-25-Year-Old-Woman-from-Zambia-With-a-N_2022_Clinical-Cases-in-Tropica/75---A-25-Year-Old-Woman-from-Zambia-With-a-N_2022_Clinical-Cases-in-Tropica.md
Working with extracted_data/76---A-55-Year-Old-Woman-from-Turkey-With-Feve_2022_Clinical-Cases-in-Tropic/76---A-55-Year-Old-Woman-from-Turkey-With-Feve_2022_Clinical-Cases-in-Tropic.md
Working with extracted_data/77---A-51-Year-Old-Female-Traveller-Returning-from-Cen_2022_Clinical-Cases-i/77---A-51-Year-Old-Female-Traveller-Returning-from-Cen_2022_Clinical-Cases-i.md
Working with extracted_data/78---A-42-Year-Old-British-Man-Living-in-Malawi_2022_Clinical-Cases-in-Tropi/78---A-42-Year-Old-British-Man-Living-in-Malawi_2022_Clinical-Cases-in-Tropi.md
Working with extracted_data/79---A-34-Year-Old-Male

In [ ]:
def embed_image_text_pairs(texts, images):
    if len(texts) != len(images):
        raise ValueError("Texts and images must have the same length.")

    # Prepare inputs separately
    text_inputs = processor(text=texts, return_tensors="pt", padding=True, truncation=True).to(device)
    image_inputs = processor(images=images, return_tensors="pt").to(device)

    # Forward pass with autocast to unify dtype
    with torch.no_grad():
        if device.startswith("cuda"):
            # On GPU, use mixed precision
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                text_embeds = model.get_text_features(**text_inputs)
                image_embeds = model.get_image_features(**image_inputs)
        else:
            # On CPU, float32
            text_embeds = model.get_text_features(**text_inputs)
            image_embeds = model.get_image_features(**image_inputs)

    # Normalize
    text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True)
    image_embeds = image_embeds / image_embeds.norm(p=2, dim=-1, keepdim=True)

    #final early fusion
    fused_emb = text_embeds * 0.2 + image_embeds * 0.8
    return fused_emb


In [ ]:
image_caption_embed = embed_image_text_pairs(caption, pil_images)

# Vector database and retrieval

In [ ]:
%pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 9.8 MB/s eta 0:00:00


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
import uuid
QDRANT_URL = "QDRANT_URL_ENDPOINT"
QDRANT_API_KEY = "QDRANT_API_KEY"
client = QdrantClient(url=QDRANT_URL,api_key=QDRANT_API_KEY)

In [ ]:
VECTOR_SIZE = 768  # vector size for jina clip embedding

client.create_collection(
    collection_name="multimodal_embeddings",
    vectors_config=models.VectorParams(
        size=VECTOR_SIZE,
        distance=models.Distance.COSINE
    )
)

True

In [ ]:
import uuid
def store_text(context, embedded_context, hashtag, embedded_hashtag):
    for each_context, each_context_emb, each_hashtag, each_hashtag_emb in zip(
        context, embedded_context, hashtag, embedded_hashtag
    ):
        client.upsert(
            collection_name="multimodal_embeddings",
            points=[
                models.PointStruct(
                    id=str(uuid.uuid4()),
                    vector=each_context_emb,  # ✅ FIXED
                    payload={
                        "type": "disease_overview",
                        "content": each_context,
                        "disease_name": each_hashtag,
                    },
                )
            ],
        )

        client.upsert(
            collection_name="multimodal_embeddings",
            points=[
                models.PointStruct(
                    id=str(uuid.uuid4()),
                    vector=each_hashtag_emb,  # ✅ FIXED
                    payload={
                        "type": "disease_name",
                        "content": each_hashtag,
                    },
                )
            ],
        )


def store_image_caption(fused_emb, caption, image_path, origin_file):
    for fused_elem,each_origin_file,each_caption in zip(fused_emb,origin_file,caption):
      if not isinstance(fused_elem, list):
          fused_elem = fused_elem.tolist()
      client.upsert(
          collection_name="multimodal_embeddings",
          points=[
              models.PointStruct(
                  id=str(uuid.uuid4()),
                  vector=fused_elem,
                  payload={
                      "type": "image_caption",
                      "caption": each_caption,
                      "orginal_file": each_origin_file,
                      "Image_path": image_path
                  }
              )
          ]
    )


In [ ]:
store_text(context1, embedded_context1, hashtag1, embedded_hashtag1)

In [ ]:
store_text(context2, embedded_context2, hashtag2, embedded_hashtag2)

In [ ]:
store_text(context3, embedded_context3, hashtag3, embedded_hashtag3)

In [ ]:
store_text(context4, embedded_context4, hashtag4, embedded_hashtag4)

In [ ]:
"""
For deleting things in Qdrant, use the following code:
"""
"""
from qdrant_client.models import Filter, FieldCondition, MatchValue
from qdrant_client.http import models as rest
client.create_payload_index(
    collection_name="multimodal_embeddings",
    field_name="type",
    field_schema=rest.PayloadSchemaType.KEYWORD
)
client.delete(
    collection_name="multimodal_embeddings",
    points_selector=models.FilterSelector(
        filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="type",
                    match=models.MatchValue(value="text")
                )
            ]
        )
    )
)
"""

UpdateResult(operation_id=7, status=<UpdateStatus.COMPLETED: 'completed'>)

# Retrieval

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue
from qdrant_client.http import models as rest
def retrieve_text_contexts(query_text, keyword = None, top_k=3, string_formatted_result = True):
  # Ensure the payload indexes are created for the correct collection (can be called multiple times safely)
  client.create_payload_index(
      collection_name="multimodal_embeddings",
      field_name="type",
      field_schema=rest.PayloadSchemaType.KEYWORD
  )
  client.create_payload_index(
      collection_name="multimodal_embeddings",
      field_name="disease_name",
      field_schema=rest.PayloadSchemaType.KEYWORD
  )
  if keyword:
    must_conditions = [
        FieldCondition(key="type", match=MatchValue(value="disease_name"))
    ]
    results = client.query_points(
      collection_name="multimodal_embeddings",
      query=embed_plain_text(keyword)[0].tolist(),
      limit=top_k,
      query_filter=Filter(must=must_conditions),
      with_payload=True,
      with_vectors=False,
      score_threshold=None
    )
    try:
      retrieved_disease_name_tag = results.points[0].payload.get("content","")
    except Exception as e:
      print(f"Warning: Could not retrieve disease name for keyword '{keyword}'. Error: {e}")
      return [] # Return empty list if no disease name found for keyword
    must_conditions = [
        FieldCondition(key="type", match=MatchValue(value="disease_overview"))
    ]
    must_conditions.append(FieldCondition(key="disease_name", match=MatchValue(value=keyword)))
    overview_results = client.query_points(
      collection_name="multimodal_embeddings",
      query=embed_plain_text(query_text)[0].tolist(),
      limit=top_k,
      query_filter=Filter(must=must_conditions),
      with_payload=True,
      with_vectors=False,
      score_threshold=None
    )
    if string_formatted_result == True:
      formatted_result = ""
      for each_result in overview_results.points:
        formatted_result += f"{each_result.payload.get('content','')}\n"
      return formatted_result
    else:
      return overview_results.points
  else:
    must_conditions = [
        FieldCondition(key="type", match=MatchValue(value="disease_overview"))
    ]
    results = client.query_points(
      collection_name="multimodal_embeddings",
      query=embed_plain_text(query_text)[0].tolist(), # Fixed: Use query_text when keyword is None
      limit=top_k,
      query_filter=Filter(must=must_conditions),
      with_payload=True,
      with_vectors=False,
      score_threshold=None
    )
    if string_formatted_result:
      formatted_result = ""
      for each_result in results.points: # Fixed: Iterate over 'results.points'
        formatted_result += f"{each_result.payload.get('content','')}\n"
      return formatted_result
    else:
      return results.points

In [ ]:
retrieve_text_contexts("Patient was a 33-year-old physician who had been working in Liberia since October 2013, during which time he had remained healthy while taking daily combination therapy with atovaquone and proguanil as prophylaxis against malaria. In April 2014, he and his team established an EVD care unit in Monrovia, and patients with confirmed EVD began arriving at this facility on June 11, 2014. On July 23, 2014, he awoke feeling febrile and fatigued; his oral temperature was 37.8°C. He reported his symptoms to colleagues and remained at home. Results on two rapid diagnostic tests for malaria (Standard Diagnostics) were negative. He started empirical malaria treatment with artemether and lumefantrine. Later that day, his oral temperature was 38.6°C, and nausea developed. He was tested for malaria by means of a rapid diagnostic test and for yellow fever, Lassa fever, and EBOV by means of semiquantitative real-time reverse-transcriptase–polymerase-chain-reaction (RT-PCR) assays, all of which were performed at the Liberian National Reference Laboratory. The results for all the tests were negative.",keyword= "Ebola", top_k =5)

'Ebola virus disease primarily occurs in tropical rainforests in sub-Saharan Africa and typically presents with fever, headache, asthenia, myalgias, abdominal pain, nausea, vomiting, and diarrhoea, with conjunctival injection and subconjunctival haemorrhage. Clinical presentation can include non-specific symptoms and rapidly progress to haemorrhage, multiple organ dysfunction, and shock. Important differential diagnoses include malaria, typhoid fever, bacterial gastroenteritis, and leptospirosis. The diagnosis is confirmed by detection of Ebola virus using ELISA antigen and PCR tests from blood or other body fluids, supported by relevant epidemiological exposure. The disease is caused by exposure to infected bat excreta or saliva, non-human primates, or person-to-person transmission in settings of inadequate universal precautions. Treatment is supportive, with antimalarials and broad-spectrum antibiotics given until the diagnosis of viral HF is confirmed.\n'

In [ ]:
from io import BytesIO
from PIL import Image, UnidentifiedImageError
import requests

def retrieve_image_caption_contexts(image_path, support_text='', top_k=5):
    # --- Step 1: Load image safely
    try:
        response = requests.get(image_path, timeout=10)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content)).convert("RGB")
    except (requests.RequestException, UnidentifiedImageError) as e:
        print(f"⚠️ Failed to load image from {image_path}: {e}")
        return {}

    # --- Step 2: Embed image + text
    try:
        fused_emb = embed_image_text_pairs([support_text], [image])
        query_vector = fused_emb[0].tolist()
    except Exception as e:
        print(f"⚠️ Embedding failed for {image_path}: {e}")
        return {}

    # --- Step 3: Query image captions
    try:
        results = client.query_points(
            collection_name="multimodal_embeddings", # Corrected collection name
            query=query_vector,
            limit=top_k,
            query_filter=Filter(must=[
                FieldCondition(key="type", match=MatchValue(value="image_caption"))
            ])
        )
    except Exception as e:
        print(f"⚠️ Query failed: {e}")
        return {}

    if not results.points:
        print(f"⚠️ No image captions found for {image_path}")
        return {}

    # --- Step 4: Build output safely
    output = {}
    for res in results.points:
        content = res.payload.get("caption", "")
        # The 'orginal_file' payload does not contain '.pdf.json' in store_image_caption, so removing the suffix add here.
        # The original code here used f"{res.payload.get('orginal_file','')}.pdf.json", which could be incorrect based on how it's stored.
        # Assuming 'orginal_file' is stored directly as a path without '.pdf.json'.
        origin_file = res.payload.get('orginal_file','')

        try:
            # Ensure payload indexes are created before querying
            client.create_payload_index(
                collection_name="multimodal_embeddings",
                field_name="orginal_file",
                field_schema=rest.PayloadSchemaType.KEYWORD
            )
            client.create_payload_index(
                collection_name="multimodal_embeddings",
                field_name="disease_name", # Assuming disease_name is a field in the payload of type 'disease_overview'
                field_schema=rest.PayloadSchemaType.KEYWORD
            )

            # Query for disease_overview related to the origin_file
            disease_name_result = client.query_points(
                collection_name="multimodal_embeddings", # Corrected collection name
                query=query_vector, # Use the fused image+text query vector for semantic relevance
                limit=1,
                query_filter=Filter(must=[
                    FieldCondition(key="orginal_file", match=MatchValue(value=origin_file)),
                    FieldCondition(key="type", match=MatchValue(value="disease_overview"))
                ]),
                with_payload=True,
                with_vectors=False
            )

            if disease_name_result.points:
                disease_name = disease_name_result.points[0].payload.get("disease_name", "Unknown disease")
            else:
                disease_name = "Unknown disease"
        except Exception as e:
            print(f"⚠️ Disease name query failed: {e}")
            disease_name = "Unknown disease"

        if content:  # Only include valid content
            output[res.id] = {
                "content": f"{disease_name}-related image: {content}",
                "Score": res.score,
                "Type": "Image_caption"
            }

    return output

In [ ]:
res = retrieve_text_contexts("Pneumocystis")

In [ ]:
res

{'4ef73449-03e1-4844-bf8a-340d14f59791': {'content': 'disease_name: Pneumocystis jirovecii pneumonia (PCP)',
  'Score': 0.7185345,
  'Type': 'Text'},
 'c733045a-d495-442d-be5f-07af5ec47512': {'content': 'Pneumocystis jirovecii pneumonia (PCP) disease_overview: P. jirovecii is a fungus found worldwide, causes Pneumocystis pneumonia (PCP) in immunocompromised individuals. Classic symptoms are sub-acute onset of malaise, non-productive cough and progressive exertional dyspnoea; fever may also be present.',
  'Score': 0.5562278,
  'Type': 'Text'},
 '524bc601-7e63-419d-ae57-59ee2a2982ec': {'content': 'disease_name: Neurocysticercosis',
  'Score': 0.545038,
  'Type': 'Text'},
 'b49e423b-c1dd-4b6a-b654-80a2ec176f45': {'content': 'disease_name: Syphilis',
  'Score': 0.519081,
  'Type': 'Text'},
 '342611b6-f3ef-4c78-ac23-5ef99532cd78': {'content': 'Pneumocystis jirovecii pneumonia (PCP) diagnosis: Diagnosis was confirmed by bronchoscopy and immunofluorescence or Grocott methenamine silver stain

In [ ]:
image_res = retrieve_image_caption_contexts("https://raw.githubusercontent.com/Springblade/Project-LLM/main/extracted_data/12---A-29-Year-Old-Man-from-The-Gambia-With-G_2022_Clinical-Cases-in-Tropica/images/_page_0_Picture_11.jpeg",
                                            "Numerous painful ulcers on the penis, scrotum and inner thigh. The ulcers are soft, tender and bleed on contact.",)

In [ ]:
image_res

{'95908498-0325-4262-a959-212afcc4c7d8': {'content': 'Chancroid-related image: Numerous painful ulcers on the penis, scrotum and inner thigh. The ulcers are soft, tender and bleed on contact.',
  'Score': 0.9999988,
  'Type': 'Image_caption'},
 '9b7b9cc1-6bfd-4b4d-8804-23e7ca9bb71d': {'content': 'Chancroid-related image: Follow-up after 1 week of antibiotic therapy. The ulcers are healing and the pain has resolved.',
  'Score': 0.79333955,
  'Type': 'Image_caption'},
 '5cc02860-1933-43f6-9f21-901e7910f8d9': {'content': 'Sporotrichosis-related image: Initial ulcerated lesion on the left hand. Note the pustular lesion near the ulcer.',
  'Score': 0.7508372,
  'Type': 'Image_caption'},
 'db2ec25f-a6c8-43de-883f-d8f2f417013a': {'content': 'Crusted Scabies-related image: Severe crusted scabies of the right axilla and chest wall.',
  'Score': 0.74894905,
  'Type': 'Image_caption'},
 '2b38b803-c8a5-43d1-94af-7caaf4200070': {'content': 'Buruli ulcer (BU)-related image: Large skin ulcer on the 

In [ ]:
import json
import re
import ast # Ensure ast is imported
def generate_answer(user_prompt_input, answer_type = "default"):
  payload = []
  from groq import Groq
  client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

  # Ensure user_prompt_input is a dictionary
  user_prompt_dict = {}
  if isinstance(user_prompt_input, str):
    user_prompt_dict = json.loads(user_prompt_input)
  elif isinstance(user_prompt_input, dict):
    user_prompt_dict = user_prompt_input
  else:
    raise ValueError("user_prompt_input must be a string or a dictionary")

  # Extract Text and ImageList, considering both top-level and nested 'Question' structures
  text_content = ""
  raw_image_list = []

  if "question" in user_prompt_dict:
      q_data = user_prompt_dict.get("question", {})
      text_content = q_data.get("Text", "")
      raw_image_list = q_data.get("ImageList", [])
  else:
      # Assume top-level if 'question' key is not present
      text_content = user_prompt_dict.get("Text", "")
      raw_image_list = user_prompt_dict.get("ImageList", [])


  if answer_type == "self-query":
    system_instruction = """ Based on the images and clinical textual description, provide a list of initial diagnosis disease. No additional comment, maximum 5 diseases separated by comma.
          Example:
          User input: Patient is 30 years old comes from Vietnam experiences sudden high fever, severe headache (especially behind the eyes), intense muscle/joint pain ("breakbone fever"), nausea, vomiting, and a skin rash
          LLM output: Dengue, Chikungunya, malaria.
    """
  elif answer_type == "direct-query":
    system_instruction = """
        Based on the images and clinical textual description. Summarise the information into a brief clinical patient condition and symptom useful for doctors to diagnose, no additional comment or suggestion.
    """
  elif answer_type == "semi-augmented-prompt":
    system_instruction = "Based on the given context, output your final diagnose(s) and provide 2-4 sentences to support your overall verdict."
  elif answer_type == "default":
    system_instruction = "
  else:
    system_instruction = ""
  if text_content.strip():
    payload.append({"type": "text", "text": text_content})

  # Process image_list
  processed_image_urls = []
  if raw_image_list:
    try:
        if isinstance(raw_image_list, str):
            processed_image_urls = ast.literal_eval(raw_image_list)
        else:
            processed_image_urls = raw_image_list
    except Exception as e:
        print(f"Error evaluating image list: {e}")
        processed_image_urls = []

  # Filter for valid URLs and limit to 5
  processed_image_urls = [url.strip() for url in processed_image_urls if isinstance(url, str) and url.startswith("http")]
  processed_image_urls = processed_image_urls[:5] # Limit to 5 images for Groq

  # Add the 'And here are the provided images: ' text once if there are images
  if processed_image_urls:
    payload.append({"type": "text", "text": "And here are the provided images: "})

  for img_url in processed_image_urls:
      payload.append({"type": "image_url", "image_url": {"url": img_url}})
      print(f"Working with url: {img_url}\n")
  response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
      {"role": "system", "content": system_instruction},
      {"role": "user", "content": payload},
    ],
    temperature=0.4,
    max_tokens=512
  )
  output_text = response.choices[0].message.content # Corrected access
  return output_text

In [ ]:
import json
user_input = {
    "Text": "Patient was a 33-year-old physician who had been working in Liberia since October 2013, during which time he had remained healthy while taking daily combination therapy with atovaquone and proguanil as prophylaxis against malaria. In April 2014, he and his team established an EVD care unit in Monrovia, and patients with confirmed EVD began arriving at this facility on June 11, 2014. On July 23, 2014, he awoke feeling febrile and fatigued; his oral temperature was 37.8°C. He reported his symptoms to colleagues and remained at home. Results on two rapid diagnostic tests for malaria (Standard Diagnostics) were negative. He started empirical malaria treatment with artemether and lumefantrine. Later that day, his oral temperature was 38.6°C, and nausea developed. He was tested for malaria by means of a rapid diagnostic test and for yellow fever, Lassa fever, and EBOV by means of semiquantitative real-time reverse-transcriptase–polymerase-chain-reaction (RT-PCR) assays, all of which were performed at the Liberian National Reference Laboratory. The results for all the tests were negative.",
    "ImageList" : [],
}
json_obj = json.dumps(user_input)
result = self_query_retrieval(json_obj, only_diagnose_result = True)

NameError: name 'self_query_retrieval' is not defined

In [ ]:
print(result)

Ebola, malaria, Lassa fever, yellow fever, viral hemorrhagic fever.


Finalize the RAG pipeline

In [ ]:
def RAG_pipeline(dataset):
  import copy
  error_sample = {}
  output_sample = copy.deepcopy(dataset)
  for sample_id, sample_value in dataset.items():
    init_val = sample_value
    modifiable_sample_value = copy.deepcopy(sample_value)
    context = ""
    print(f"processing sample: {sample_id}")
    try:
      init_response = generate_answer(sample_value, answer_type = "self-query")
      diagnose_list = init_response.split(",")
      for each_diagnose in diagnose_list:
        temp = retrieve_text_contexts(each_diagnose,string_formatted_result=True)
        context += f"{temp}\n"
      augmented_question = f"The initial diagnose include these disease: {init_response} and its relevant information: {context}"
      #Augment the question in the dataset with final_query_request
      modifiable_sample_value["question"]["Text"] = sample_value["question"]["Text"] + augmented_question
      generated_answer = generate_answer(modifiable_sample_value, answer_type = "semi-augmented-prompt")



      output_sample[sample_id]["contexts"] = context
      output_sample[sample_id]["answer"] = generated_answer
    except Exception as e:
      print(f"Error processing sample {sample_id}: {e}")
      error_sample[sample_id] = str(e)
  return output_sample,error_sample


In [ ]:
Tst = {
  "1": {
    "question": {
      "Text": "A 28-year-old man who recently returned from rural West Africa presents with acute onset of high fever, severe headache, myalgia, retro-orbital pain, and nausea. On examination, he has a maculopapular rash and mild thrombocytopenia on laboratory testing. What is the most likely diagnosis?",
      "ImageList": []
    },
    "ground_truth": "Dengue fever",
    "answer": ""
  },

  "2": {
    "question": {
      "Text": "A 35-year-old woman living in a tropical rainforest region presents with intermittent fever, chills, night sweats, and progressive fatigue over several weeks. Physical examination reveals splenomegaly and pallor. Blood smear shows parasitized red blood cells with ring forms. What is the most likely diagnosis?",
      "ImageList": []
    },
    "ground_truth": "Malaria",
    "answer": ""
  }
}


In [ ]:
O,E = RAG_pipeline(Tst)

processing sample: 1
processing sample: 2


In [ ]:
O

{'1': {'question': {'Text': 'A 28-year-old man who recently returned from rural West Africa presents with acute onset of high fever, severe headache, myalgia, retro-orbital pain, and nausea. On examination, he has a maculopapular rash and mild thrombocytopenia on laboratory testing. What is the most likely diagnosis?',
   'ImageList': []},
  'ground_truth': 'Dengue fever',
  'answer': "Based on the patient's symptoms and travel history, the most likely diagnosis is **Yellow Fever**. The patient's presentation with acute onset of high fever, severe headache, myalgia, retro-orbital pain, nausea, maculopapular rash, and mild thrombocytopenia are consistent with Yellow Fever. Additionally, the patient's recent travel to rural West Africa, a region where Yellow Fever is endemic, increases the likelihood of this diagnosis. Yellow Fever is a viral hemorrhagic fever that can cause liver failure, characterized by fever, jaundice, bleeding, encephalopathy, and seizures, which aligns with the pat

#Evaluation

In [ ]:
import os
import json
import requests
# Step 1: Download JSON files from GitHub (16 cases from medical journal)
api_url = "https://raw.githubusercontent.com/Springblade/Project-LLM/main/Diagnose/Diagnose_dataset.json"

response = requests.get(api_url)
response.raise_for_status()
Test_dataset = response.json()

In [ ]:
Test_dataset

{'1': {'question': {'Text': 'The first patient was a 33-year-old physician who had been working in Liberia since October 2013, during which time he had remained healthy while taking daily combination therapy with atovaquone and proguanil as prophylaxis against malaria. In April 2014, he and his team established an EVD care unit in Monrovia, and patients with confirmed EVD began arriving at this facility on June 11, 2014. On July 23, 2014, he awoke feeling febrile and fatigued; his oral temperature was 37.8°C. He reported his symptoms to colleagues and remained at home. Results on two rapid diagnostic tests for malaria (Standard Diagnostics) were negative. He started empirical malaria treatment with artemether and lumefantrine. Later that day, his oral temperature was 38.6°C, and nausea developed. He was tested for malaria by means of a rapid diagnostic test and for yellow fever, Lassa fever, and EBOV by means of semiquantitative real-time reverse-transcriptase–polymerase-chain-reaction

In [ ]:
Output_list, Error_list = RAG_pipeline(Test_dataset)

processing sample: 1
Working with url: https://raw.githubusercontent.com/Springblade/Project-LLM/refs/heads/main/Diagnose/1.jpg

Working with url: https://raw.githubusercontent.com/Springblade/Project-LLM/refs/heads/main/Diagnose/2.jpg

Working with url: https://raw.githubusercontent.com/Springblade/Project-LLM/refs/heads/main/Diagnose/1.jpg

Working with url: https://raw.githubusercontent.com/Springblade/Project-LLM/refs/heads/main/Diagnose/2.jpg

processing sample: 2
Working with url: https://raw.githubusercontent.com/Springblade/Project-LLM/refs/heads/main/Diagnose/1.jpg

Working with url: https://raw.githubusercontent.com/Springblade/Project-LLM/refs/heads/main/Diagnose/2.jpg

Working with url: https://raw.githubusercontent.com/Springblade/Project-LLM/refs/heads/main/Diagnose/1.jpg

Working with url: https://raw.githubusercontent.com/Springblade/Project-LLM/refs/heads/main/Diagnose/2.jpg

processing sample: 3
Working with url: https://raw.githubusercontent.com/Springblade/Project-L

In [ ]:
Output_list

{'1': {'question': {'Text': 'The first patient was a 33-year-old physician who had been working in Liberia since October 2013, during which time he had remained healthy while taking daily combination therapy with atovaquone and proguanil as prophylaxis against malaria. In April 2014, he and his team established an EVD care unit in Monrovia, and patients with confirmed EVD began arriving at this facility on June 11, 2014. On July 23, 2014, he awoke feeling febrile and fatigued; his oral temperature was 37.8°C. He reported his symptoms to colleagues and remained at home. Results on two rapid diagnostic tests for malaria (Standard Diagnostics) were negative. He started empirical malaria treatment with artemether and lumefantrine. Later that day, his oral temperature was 38.6°C, and nausea developed. He was tested for malaria by means of a rapid diagnostic test and for yellow fever, Lassa fever, and EBOV by means of semiquantitative real-time reverse-transcriptase–polymerase-chain-reaction

In [ ]:
Output_list

{'context': "Acute bacterial meningitis primarily occurs in individuals with immunocompromised status, such as HIV-positive patients, and presents with symptoms like high fever, rapid decline in consciousness, seizures, stiff neck, and altered mental status. Clinically, it is characterized by cerebrospinal fluid (CSF) analysis showing pleocytosis, increased protein, and decreased glucose. A lumbar puncture is necessary to obtain a diagnosis, and antibiotics should be administered as soon as possible. Important differential diagnoses include tuberculous meningitis, cryptococcal meningitis, and other bacterial causes like Streptococcus pneumoniae and Neisseria meningitidis.\nTuberculous meningitis primarily occurs in settings with high tuberculosis prevalence and is characterized by florid gelatinous exudate in the subarachnoid space, causing symptoms such as fever, headache, cranial nerve palsies, and altered consciousness. Clinically, it presents as a subacute illness with systemic sym

In [ ]:
#def save_dict_to_json(file_name: str, data: dict, base_dir: str = "/content/drive/MyDrive/TextSummary"):
save_dict_to_json("RAG_LLaMA_output_test1.json", Output_list, base_dir="/content/drive/MyDrive/TestResult")


JSON saved successfully to: /content/drive/MyDrive/TestResult/RAG_LLaMA_output_test1.json


Non RAG generated answer

In [ ]:
def generate_non_rag_answer(dataset):
  import copy
  error_sample = {}
  output_sample = copy.deepcopy(dataset)
  for sample_id, sample_value in dataset.items():
    print(f"processing sample: {sample_id}")
    try:
      generated_answer = generate_answer(sample,answer_type = "semi-augmented-prompt")
      output_sample[sample_id]["answer"] = generated_answer
      output_sample[sample_id]["contexts"] = ""
    except Exception as e:
      print(f"Error processing sample {sample_id}: {e}")
      error_sample[sample_id] = str(e)
  return output_sample,error_sample